In [6]:
!pip install vosk

In [28]:
from vosk import Model, KaldiRecognizer, SpkModel
import IPython.display as ipd
import numpy as np
import wave
import json
import sys
import re
import os


def find_person(array):
    database = np.array(list(users.values()))
    distances = []
    
    for i in database:
        distances.append(cosine_dist(array, i))
    min_dist = min(distances)
    
    if min_dist > 0.12:
        return 'UNKNOWN PERSON'
    
    key = distances.index(min_dist)
    return list(users.keys())[key]


def punctuation(text):
    text = re.sub(' а ', ', а ', text)
    text = re.sub(' но ', ', но ', text)
    text = re.sub(' что ', ', что ', text)
    text = re.sub(' чтобы ', ', чтобы ', text)
    text = re.sub(' например ', ', например,  ', text)
    text = re.sub(' да ', ', да  ', text)
    text = re.sub(' как ', ', как  ', text)
    text = re.sub(' зато ', ', зато  ', text)
    text = re.sub(' кроме того ', ', кроме того,  ', text)
    text = re.sub(' здравствуйте ', ' здравствуйте,  ', text)
    return text.capitalize() + '.'


def cosine_dist(x, y):
    nx = np.array(x)
    ny = np.array(y)
    return 1 - np.dot(nx, ny) / np.linalg.norm(nx) / np.linalg.norm(ny)



def init_weights(persons):
    """ Функция присваивает каждому человеку вектор нулей"""
    users = {}
    for i in persons:
        users.update({i: np.zeros(128)})
    return users

persons = ['UNKNOWN PERSON', 'timur', 'danya', 'anton']
users = init_weights(persons)

# модель🙈, делающая разбиение речи на текст
model_path = "vosk-model-ru-0.10"
# модель 🙉, делающая разбиение текста на диалоги
spk_model_path = "vosk-model-spk-0.3"
# путь к войсу
PATH = 'sample/ffmpeg_output_3.wav'


wf = wave.open(PATH, "rb")
model = Model(model_path)
spk_model = SpkModel(spk_model_path)
rec = KaldiRecognizer(model, spk_model, wf.getframerate())


while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print ("{}: ".format(find_person(res['spk'])), punctuation(res['text']))


res = json.loads(rec.FinalResult())
print ("{}: ".format(find_person(res['spk'])), res['text'])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


UNKNOWN PERSON:  Я буду уборщицей.
UNKNOWN PERSON:  Короче я буду отсчитывать упор я уборщица.
UNKNOWN PERSON:  Здравствуйте клава михайловна я вас пригласил, чтобы поговорить с вами на тему того, как  плохо вы помыли полы в туалете объясните пожалуйста почему вы не вытерли.
UNKNOWN PERSON:  А я, что я ничто я сидела себе спокойно смотрела свои сериалы.
UNKNOWN PERSON:  Вот ну в смысле не сериалы я смотрела в воду в унитазе и все было хорошо гадать на кофейной гуще.
UNKNOWN PERSON:  Ну там другая гуще.
UNKNOWN PERSON:  Так ладно я думаю это пойти.
UNKNOWN PERSON:  


In [29]:
# сам диалог
ipd.Audio(PATH)

### Как видим, ни один из участников диалога не распознан. 🙉

Обучим на отдельно взятых записях:

In [25]:
def train_on_records(users):
    matrices = []
    for i in list(users.keys()):
        print(f'\nОбучение на голосе {i}')
        print('='*10)
        print()
        PATH = 'sample/'+i+'-2.wav'
        wf = wave.open(PATH, "rb")
        model = Model(model_path)
        spk_model = SpkModel(spk_model_path)
        rec = KaldiRecognizer(model, spk_model, wf.getframerate())
        
        matrices = []
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                res = json.loads(rec.Result())
                
                print ("{}: ".format(find_person(res['spk'])), punctuation(res['text']))
                matrices.append(res['spk'])

        res = json.loads(rec.FinalResult())

        print ("{}: ".format(find_person(res['spk'])), res['text'])
        matrices = np.array(matrices)
        users[i] = matrices.mean(axis=0)

persons = ['timur', 'danya', 'anton']
users = init_weights(persons)
train_on_records(users)


Обучение на голосе timur



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


timur:  Ты.
timur:  Демократическая россия во главе с не менее демократическим президентом и его ветками умело используют во внутренней и внешней политики страны политику тайной дипломатии политика.
timur:  Который начал при виде крючок ещё во времена перестройки пристройка преподносилась советского народа, как  запутано нем добытого просто предоставлением больших экономических возможностей, как  расширение демократических прав и свобод вся горбачевская фразеологии такая, как  углубить расширить глаз то есть новые мыши.
timur:  Мышление его галка галка не раскрывать перед нами не только провинциал на политическую клоун глубокого сошлись на голове оказавшемуся, да  у руля огромной державы державы, что горбачева выходцев из ставропольского края неожиданно перебравшийся в кремлёвские палаты, но не научившаяся.
timur:  Даже правильно говорить по-русски пышно принимали великобритании франции сша головы этих провинциалов нетрудно было жить прелестями западной жизни и особым вниманием к ним л

In [ ]:
# сам диалог
ipd.Audio(PATH)

### После обучения, запустим еще раз алгоритм распознавания по голосу

In [26]:
# модель🙈, делающая разбиение речи на текст
model_path = "vosk-model-ru-0.10"
# модель 🙉, делающая разбиение текста на диалоги
spk_model_path = "vosk-model-spk-0.3"
# путь к войсу
PATH = 'sample/ffmpeg_output_3.wav'


wf = wave.open(PATH, "rb")
model = Model(model_path)
spk_model = SpkModel(spk_model_path)
rec = KaldiRecognizer(model, spk_model, wf.getframerate())


while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print ("{}: ".format(find_person(res['spk'])), punctuation(res['text']))


timur:  Я буду уборщицей.
danya:  Короче я буду отсчитывать упор я уборщица.
danya:  Здравствуйте клава михайловна я вас пригласил, чтобы поговорить с вами на тему того, как  плохо вы помыли полы в туалете объясните пожалуйста почему вы не вытерли.
timur:  А я, что я ничто я сидела себе спокойно смотрела свои сериалы.
timur:  Вот ну в смысле не сериалы я смотрела в воду в унитазе и все было хорошо гадание на кофейной гуще.
timur:  Ну там другая гуще.
UNKNOWN PERSON:  Так ладно я думаю это пойти.


In [ ]:
ipd.Audio(PATH)

### Выводы: